# Building Data Genome Project 2.0
## Cleaned datasets

Biam! (pic.biam@gmail.com)

In [23]:
# data and numbers
import numpy as np
import pandas as pd
import datetime as dt
import datetime

In [3]:
path_raw = "..//data//meters//raw//"
path_cleaned = "..//data//meters//cleaned//"
path_anom = "..//data//meters//screening//anomalies//"

# Introduction

In this notebooks cleaned meters data-set will be created. Outliers in the raw meters dataset were detected using the [Seasonal Hybrid ESD (S-H-ESD)](https://github.com/twitter/AnomalyDetection) developed by Twitter. This was implemented in R language, the process can be found [here](https://github.com/buds-lab/building-data-genome-project-2/blob/master/notebooks/02_Anomaly-detection.ipynb).

# Functions

In [4]:
meter = pd.read_csv(path_raw + "irrigation.csv")

In [11]:
meter = meter.melt(id_vars="timestamp", var_name="building_id",value_name="meter_reading")

In [15]:
meter["meter"] = "irrigation"

In [16]:
meter.head()

,timestamp,building_id,meter_reading,meter
0,2016-01-01 00:00:00,Panther_lodging_Dianna,0.0,irrigation
1,2016-01-01 01:00:00,Panther_lodging_Dianna,0.0,irrigation
2,2016-01-01 02:00:00,Panther_lodging_Dianna,0.0,irrigation
3,2016-01-01 03:00:00,Panther_lodging_Dianna,0.0,irrigation
4,2016-01-01 04:00:00,Panther_lodging_Dianna,0.0,irrigation


In [ ]:
zero_gaps = find_zero_gaps(meter)

In [19]:
# Gaps longer than 24 hs
df_gaps = zero_gaps[zero_gaps["cnt"]>=23].reset_index(drop=True)

In [80]:
def find_zero_gaps(df):
    status = {}
    gaps = []
    total_rows = df.timestamp.count()
    pos = 0
    for i, row in df.iterrows():
        # Initialize status for this meter
        bmid = str(row["building_id"]) + "_" + str(row["meter"])
        if bmid not in status:
            status[bmid] = {
                "building_id": row["building_id"],
                "meter": row["meter"],
                "start": None,
                "end": None,
                "count": 0,
                "last_ts": None,
            }
        meter_status = status[bmid]
        is_zero = row["meter_reading"] == 0
        if is_zero:
            if status[bmid]["start"] is None:  # first zero detected
                status[bmid]["start"] = row["timestamp"]
                status[bmid]["count"] = 0
            else:
                status[bmid]["count"] = status[bmid]["count"] + 1
        else:
            if status[bmid]["start"] is not None:  # End of gap
                status[bmid]["end"] = row["timestamp"]
                gaps.append(
                    (
                        row["building_id"],
                        row["meter"],
                        status[bmid]["start"],
                        status[bmid]["end"],
                        status[bmid]["count"],
                    )
                )
                status[bmid]["start"] = None
                status[bmid]["end"] = None
                status[bmid]["count"] = 0
        status[bmid]["last_ts"] = row["timestamp"]
        progress = round((pos / total_rows) * 100, 2)
        if pos % 10000 == 0:
            #print("[find_gaps] progress: %" + str(progress))
            print(f"\rProgress: {progress}%", end="", flush=True)
        pos = pos + 1

    #print("[find_gaps] progress: %" + str(progress))
    print(f"\rProgress: {progress}%", end="", flush=True)
    # close trailing gaps

    for bmid in status:
        s = status[bmid]
        if (s["start"] is not None) & (s["end"] is None):  # Trailing gap
            gaps.append(
                (s["building_id"], s["meter"], s["start"], s["last_ts"], s["count"])
            )
    df_gaps = pd.DataFrame.from_dict(gaps)
    df_gaps.rename(
        columns={0: "building_id", 1: "meter", 2: "ts_from", 3: "ts_to", 4: "cnt"},
        inplace=True,
    )
    return df_gaps

In [74]:
def removeZeroGaps(meter_df, df_gaps):
    meter_df["zero"] = 0
    #for i in list(range(len(df_gaps) - 1)):
    for i in df_gaps.index:
        # loop progress
        percentage = round(((i+1) / len(df_gaps)) * 100, 2)
        # For the gap selected, mark rows with 1
        meter_df.loc[
            (meter_df["building_id"] == df_gaps["building_id"][i])
            & (
                (meter_df["timestamp"] >= df_gaps["ts_from"][i])
                & (meter_df["timestamp"] < df_gaps["ts_to"][i])
            ),
            "zero",
        ] = 1
        # print progress
        print(f"\rProgress: {percentage}%", end="", flush=True)

    return meter_df

https://stackoverflow.com/questions/45281597/counting-number-of-consecutive-zeros-in-a-dataframe

In [65]:
# Count consecutive zeros in building
ts = test # building column
#count
ts2 = 1-ts
tsgroup = ts.cumsum()
count0 = ts2.groupby(tsgroup).transform("count")

In [67]:
df = pd.DataFrame({"bdg":ts,"count0":count0})

In [76]:
ix = df[(df.bdg == 0) & (df.count0 > 24)].index

In [77]:
meter.isna().sum()

timestamp                       0
Panther_lodging_Dianna       2224
Panther_lodging_Dean         1072
Panther_lodging_Cora         9985
Panther_parking_Lorriane     2721
Panther_parking_Asia          620
Panther_lodging_Kara         6330
Panther_lodging_Drew           67
Panther_lodging_Ricky          56
Panther_parking_Clarence      951
Panther_parking_Adela        3587
Panther_lodging_Heather       952
Panther_office_Daina         1396
Panther_lodging_Arlene         56
Panther_office_Catherine      342
Panther_other_Bethel          863
Panther_lodging_Paulette       56
Panther_other_Iva            1093
Panther_education_Richard     760
Panther_parking_Jody          864
Panther_office_Woodrow        548
Panther_education_Aurora      766
Panther_office_Kristen       4915
Panther_office_Antonette      200
Panther_office_Valarie       3227
Panther_lodging_Georgie      3191
Panther_lodging_Shelia        900
Panther_lodging_Juliet         65
Panther_parking_Stanley      2783
Panther_lodgin

In [4]:
# This function removes outliers and 24 hours zero readings
def removeBadData(df, metername):

    # load anomalies df
    df_anom = pd.read_csv(path_anom + metername + "_anoms.csv")
    # Transform timestamp to datetime object type
    df_anom["timestamp"] = pd.to_datetime(
        df_anom["timestamp"], format="%Y-%m-%d %H:%M:%S"
    )
    # Remove timezone offset at the end of timestamp
    df_anom["timestamp"] = df_anom.timestamp.apply(lambda d: d.replace(tzinfo=None))
    # Set index
    df_anom = df_anom.set_index("timestamp")

    # Remove outliers
    outliers = df_anom.copy()
    # replace not null values with 9999 (outliers)
    outliers[outliers.isna() == False] = 9999
    # Update df with outliers data
    df.update(outliers)
    # Remove outliers
    for datapoint in df.columns:
        df[datapoint] = df[datapoint][df[datapoint] != 9999]

    # Remove zero gaps
    # Calculate daily average and aggregate data
    df_daily = df.resample("D").mean()
    # De-aggreate data asigning daily mean to each hour
    df_hourly = df_daily.resample("H").fillna(method="ffill")

    ## This dataset ends on 2017-12-31 00:00:00. Our meter dataset ends on 2017-12-31 23:00:00.##
    ## This is solved in the following code ##

    # Last row of df_hourly to copy values
    sample = df_hourly[df_hourly.index == "2017-12-31 00:00:00"]
    # Dataframe
    rng = pd.DataFrame(
        index=pd.date_range("2017-12-31 01:00:00", periods=23, freq="H"),
        columns=df.columns,
    )
    appdf = (
        sample.append(rng)
        .fillna(method="ffill")
        .drop(pd.Timestamp("2017-12-31 00:00:00"))
    )
    # Append
    df_hourly = df_hourly.append(appdf)

    # Delete zero values during whole day
    for datapoint in df_hourly.columns:
        df[datapoint] = df[datapoint][df_hourly[datapoint] > 0]

    del (df_anom, outliers, df_daily, df_hourly)

    return df

# Export cleaned datasets

In [8]:
metername = ["electricity","water","chilledwater","hotwater","gas", "steam","solar","irrigation"]

for meter in metername:    
    # load data
    df = pd.read_csv(path_raw + meter + ".csv") 
    # Transform timestamp to datetime object type
    df["timestamp"] = pd.to_datetime(df["timestamp"], format='%Y-%m-%d %H:%M:%S')
    # Set index
    df = df.set_index("timestamp")
    
    # Remove bad data
    df_clean = removeBadData(df, meter)
    df_clean.to_csv(path_cleaned + meter + "_cleaned.csv")